In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a P2TR transaction [wip]

## TODO - Tweak key and include a script path

In this section we'll create a P2TR transaction from scratch in python. This section assumes familiarity with legacy or segwit transactions.

## Reading
- https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- https://bitcoinops.org/en/preparing-for-taproot/#from-p2wpkh-to-single-sig-p2tr
- https://bitcoin.stackexchange.com/questions/111098/what-is-the-script-assembly-and-execution-in-p2tr-spend-spend-from-taproot

### Create a P2TR UTXO

In order to create a transaction spending from a P2TR UTXO, we'll first need to create the UTXO that is locked with a p2tr script. To do that, we'll define a private key, use it to generate a pubkey, then convert the pubkey to a p2tr address.

#### Create a P2TR pubkey
So far the public keys we have used in bitcoin were 33 bytes long. The last 32 bytes corresponded to the x coordinate of the point on the curve, and the first byte corresponded to whether the y coordinate was odd or even.

Now with taproot public keys, the keys are just the x coordinate, which makes it only 32 bytes long. To generate a public key from a private key we'll use code from the bip 340 python reference code imported as `tr`. A consequence of this is that every public key has two corresponding private keys.

In [2]:
privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey = tr.pubkey_gen(privkey)
print("Pubkey: ", pubkey.hex())
print("Note the length of pubkeys used for taproot: ", len(pubkey))

Pubkey:  4f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa
Note the length of pubkeys used for taproot:  32


In [3]:
# For comparison, here is the function we used to generate pubkeys for legacy and segwit v0 transactions
pubkey_legacy = privkey_to_pubkey(privkey)
print("33 byte legacy/segwitv0 pubkey: ", pubkey_legacy.hex())

# Taproot pubkeys are the same as legacy pubkeys but without the first byte.
assert(pubkey == pubkey_legacy[1:])

33 byte legacy/segwitv0 pubkey:  034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa


#### Tweaking the internal pubkey and secret key

TODO

#### Convert the pubkey to a scriptPubkey and P2TR address

P2TR Addresses are segwit V1 transactions. The first byte of the script pubkey is `OP_1` (`0x51`), followed by `0x20` to push the 32-byte public key. Note that unlike previous transaction types, we are including the direct public key rather than the hash of it, or a script.

The function `spk_to_bech32` converts the script pubkey to a bech32m address. The function differentiates between segwit v0 and v1 scriptPubkeys by inspecting the first byte.

In [4]:
spk = bytes.fromhex("5120") + pubkey
p2tr_addr = spk_to_bech32(spk, 'regtest')
print("P2TR bech32m address: ", p2tr_addr)

P2TR bech32m address:  bcrt1pfu64hh9hes90w2808n8tjc2ajp5yhddjef0ctx4s7zmsgp6cwx4q344m0j


In [5]:
# Setup bitcoind and fund the address
setup_regtest_bitcoind()
txid_to_spend, index_to_spend = fund_address(p2tr_addr, 2.001)
print(f"UTXO: {txid_to_spend}, {index_to_spend}")

UTXO: 0cdbad7f63792c1273bd68b59567664c20620f1ede54c73e68159df41433a252, 0


Here we can take a look at the transaction funding our P2TR "`witness_v1_taproot`" output. 

In [6]:
original_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_to_spend)
print("original_tx: ", original_tx)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + original_tx)
print(decoded)

original_tx:  02000000000101570983a9d34704d0e540e63ea41c4f5a3a2ca8f371b6c84c2adfd3f4c433e5290000000000fdffffff02a048ed0b000000002251204f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa7c9c181e01000000225120b7346b53b2107561d64b3b97713abba4faea4f6d05e54ecab3652838dab1b15e0247304402205a12eb720ccfd9d051e9df89baa2a7a5d99fd607fa6303958bab50a141d5631e022066194a1652b04d16ef3010031d4e0727e2405be53ab7a18058323e0a681b8d82012102692907c36c7e0ea90b505ea922a62450a5f95f6307049d9356b263f7e12d976665000000
{
  "txid": "0cdbad7f63792c1273bd68b59567664c20620f1ede54c73e68159df41433a252",
  "hash": "f9127f143845b38eddc0f6cfe1f7c9bb5eeaa742a0f3f228014241a97263c9a4",
  "version": 2,
  "size": 246,
  "vsize": 165,
  "weight": 657,
  "locktime": 101,
  "vin": [
    {
      "txid": "29e533c4f4d3df2a4cc8b671f3a82c3a5a4f1ca43ee640e5d00447d3a9830957",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402205a12eb720ccfd9d051e9df89baa2a

## Spending a P2TR UTXO using the 'key path'

Now that we have some funds locked up in a P2TR utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`. We can decode the scriptPubkey from the address. For more on this step, refer to the previous chapters.

In [7]:
receiver_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'

hrp = 'bcrt'
witver, witprog = bech32.decode(hrp, receiver_address)
pubkey_hash = bytearray(witprog)
receiver_spk = bytes.fromhex("0014") + pubkey_hash

### Create an unsigned P2TR transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [8]:
# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty.

In [9]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000152a23314f49d15683ec754de1e0f62204c666795b568bd73122c79637faddb0c0000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [10]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
print(decoded)

{
  "txid": "cda3848b630f2f7e03e72c50a90078386b77e882d3b4a4cff22ca5996b904483",
  "hash": "cda3848b630f2f7e03e72c50a90078386b77e882d3b4a4cff22ca5996b904483",
  "version": 2,
  "size": 113,
  "vsize": 113,
  "weight": 452,
  "locktime": 0,
  "vin": [
    {
      "txid": "0cdbad7f63792c1273bd68b59567664c20620f1ede54c73e68159df41433a252",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw",
        "type": "witness_v0_keyhash"
      }
    },
    {
      "value": 0.50000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "0 531260aa2a199e228c537dfa42c82bea2c7c1f4d",
 

#### Create the sighash
TODO - create the sighash using https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#common-signature-message


#### Changes with Taproot signatures

The message to get signed in taproot transactions is specified in [BIP341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#common-signature-message). Below we will sign the transaction manually according to the BIP, but some notable changes are:

- Hashes that go into the signature message use SHA256, rather than double SHA256 (HASH256). There's no gain in doing two rounds as there's no secret data, therefore a single round is more efficient.
- Signature message commits to scriptPubkeys and total amount. This prevents offline signers being lied to about where transactions are being sent to and the fee.
- The sighash is calculated using a tagged hash "hash<sub>TapSighash</sub>()". For more on tagged hashes, see the intro to taproot chapter. 

#### SIGHASH_DEFAULT
Taproot introduces a new type of sighash called `SIGHASH_DEFAULT`. The hash type is denoted by `0x00` and signs over the same fields as `SIGHASH_ALL`, except the sighash flag is not appended to the signature, saving one byte.

In [11]:
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0000 0000")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(2.001 * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

Now we'll create the sighash using a tagged hash with `"TapSighash"` as the string. The tagged hash created by prepending SHA256("TapSighash") twice to the message, then hashing the whole thing with SHA256.

In [12]:
tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)
print(sighash.hex())

6b6d2c9c010701ddb019b66e29ecbbd828f93a0b402a8fe26bb22aba7ba3472f


Great, now we are ready to create our schnorr signature! In addition to the message and the private key, we'll include a value for `auxilary randomness`. This value is used to help create a _synthetic nonce_ for the signature. The advantages of this method for nonce selection are described in this [stack exchange answer](https://bitcoin.stackexchange.com/questions/95762/k-selection-for-schnorr-signatures/95763#95763).

For our example we'll just use all zeros (32 bytes).

In [13]:
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")

We'll use the `schnorr_sign` code from the [BIP340 reference code](https://github.com/bitcoin/bips/blob/master/bip-0340/reference.py) to create our schnorr signature. Since we are using `SIGHASH_DEFAULT` (`0x00`), we will not append the sighash type to the signature.

In [14]:
signature = tr.schnorr_sign(sighash, privkey, aux_rand)

# Only append the sighash type if it is not SIGHASH_DEFAULT (0x00)
if hash_type != b"\x00":
    signature += hash_type
    
signature.hex()

'6f8d5d7ad2bdb17c520075083a333c096e928355d1d30bc027267f8ad2ed4a585bee6e8103b96f9d83092f0265b736c9e4af8ecc04c14d4f9c11e170bd863634'

Now all we need to do is add the signature to the witness field.

In [15]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + pushbytes(signature)
)
print(witness.hex())

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

01406f8d5d7ad2bdb17c520075083a333c096e928355d1d30bc027267f8ad2ed4a585bee6e8103b96f9d83092f0265b736c9e4af8ecc04c14d4f9c11e170bd863634
signed transaction:  0200000000010152a23314f49d15683ec754de1e0f62204c666795b568bd73122c79637faddb0c0000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d01406f8d5d7ad2bdb17c520075083a333c096e928355d1d30bc027267f8ad2ed4a585bee6e8103b96f9d83092f0265b736c9e4af8ecc04c14d4f9c11e170bd86363400000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [16]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex() + "\"]'")
print(new_tx_txid)

[
  {
    "txid": "cda3848b630f2f7e03e72c50a90078386b77e882d3b4a4cff22ca5996b904483",
    "wtxid": "c6e90402f30ca7b9bf6da0501cd1b64785f628fd642cf7b8c246cfd8b22da20b",
    "allowed": true,
    "vsize": 130,
    "fees": {
      "base": 0.00100000
    }
  }
]


We can decode the serialized transaction using ```decoderawtransaction```.

In [17]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
print(decoded)

{
  "txid": "cda3848b630f2f7e03e72c50a90078386b77e882d3b4a4cff22ca5996b904483",
  "hash": "c6e90402f30ca7b9bf6da0501cd1b64785f628fd642cf7b8c246cfd8b22da20b",
  "version": 2,
  "size": 181,
  "vsize": 130,
  "weight": 520,
  "locktime": 0,
  "vin": [
    {
      "txid": "0cdbad7f63792c1273bd68b59567664c20620f1ede54c73e68159df41433a252",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "6f8d5d7ad2bdb17c520075083a333c096e928355d1d30bc027267f8ad2ed4a585bee6e8103b96f9d83092f0265b736c9e4af8ecc04c14d4f9c11e170bd863634"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw",
        "type"

In [19]:
# stop bitcoin core
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'

## Quiz


 ## Answers
    

## Exercise
